## Integration 

The definite integral of a polynomial function on a finite domain $[a,b]$ can be computed very accurately via the _Fundamental Theorem of Calculus_, through the module `numpy.polynomial`.  For instance, to calculate the integral of the polynomial $p(x) = x^5$ on the interval $[-1,1]$, we could issue:


In [ ]:
import numpy as np

p = np.poly1d([1,0,0,0,0,0])
print p
print p.integ()

p.integ()(1.0) - p.integ()(-1.0)

In [ ]:
from sympy import integrate, symbols

x, y = symbols('x y', real=True)

print integrate(x**5, x)

print integrate(x**5, (x, -1, 1))

Let us try something more complicated.  The definite integral of $f(x) = e^{-x}  sin(x)$ on the interval $[0,1]$:

In [ ]:
from sympy import N, exp as Exp, sin as Sin

print integrate(Exp(-x) * Sin(x), x)

print integrate(Exp(-x) * Sin(x), (x, 0, 1))

print N(_)

In [ ]:
print integrate(Sin(x) / x, x)

print integrate(Sin(x) / x, (x, 0, 1))

print N(_)

print integrate(x**(1/x), (x, 0, 1))

In [ ]:
from sympy import oo

integrate(Exp(-x**2), (x,0,+oo))

In [ ]:
integrate(Exp(-x**2-y**2), (x, -oo, +oo), (y, -oo, +oo))

In [ ]:
print integrate(Sin(x)**Sin(x), x)

print integrate(Sin(x)**Sin(x), (x, 0, 1))

In [ ]:
from scipy.integrate import cumtrapz, simps

def f(x): return x**5

nodes = np.linspace(-1, 1, 100)

print simps(f(nodes), nodes)

print cumtrapz(f(nodes), nodes)

In [ ]:
cumtrapz(f(nodes), nodes)[-1]

In [ ]:
from scipy.integrate import newton_cotes

coefficients, absolute_error = newton_cotes(3, equal=True)
nodes = np.linspace(-1, 1, 4)
print coefficients

In [ ]:
integral = (coefficients * f(nodes)).sum()
print integral

In [ ]:
from math import fsum

integral = fsum(coefficients * f(nodes))
print integral

In [ ]:
from scipy.integrate import romb

nodes = np.linspace(-1, 1, 65)

romb(f(nodes), dx=1./32)

In [ ]:
romb(f(nodes), dx=1./32, show=True)

In [ ]:
from scipy.integrate import romberg

romberg(f, -1, 1, show=True)

In [ ]:
from scipy.integrate import quadrature, fixed_quad

value, absolute_error = quadrature(f, -1, 1, tol=1.49e-8)
print value

In [ ]:
value, absolute_error = fixed_quad(f, -1, 1, n=4)
print value                                     # four nodes

In [ ]:
from scipy.integrate import quad

value, absolute_error = quad(f, -1, 1)
print value

In [ ]:
value, absolute_error, info = quad(f, -1, 1, full_output=True)

info.keys()

In [ ]:
print "{0} function evaluations".format(info['neval'])

print "Domain broken into {0} subintervals".format(info['last'])

In [ ]:
np.set_printoptions(precision=2, suppress=True)

print info['rlist']  # integral approximations on the subintervals

In [ ]:
print info['elist']  # absolute error estimates on the subintervals

In [ ]:
print info['alist']   # left end points of the subintervals 

In [ ]:
print info['blist']   # right end points of the subintervals 

> It is possible to impose a different number of Chebyshev moments to be used.  We do so with the optional parameter `maxp1`, which imposes an upper bound to this number (rather than fixing it, for optimal results). 

Oscillatory integrals of the form $f(x) \cos(wx)$ or $f(x) \sin(wx)$, even when $f(x)$ is smooth, are especially tricky.  The integrator `quad` can tackle these integrals by calling the routine `QAWO` in `QUADPACK`.  We do so by specifying the arguments `weight='cos'` or `weight='sin'`, with `wvar=w`.

For example, for the integral of $g(x) = e^x \sin(x)$ on the interval $[-10,10]$, we compare this method with a basic `quad`:

In [ ]:
def f(x): return np.sin(x) * np.exp(x)
g = np.exp

print quad(g, -10, 10, weight='sin', wvar=1)

print quad(f, -10, 10)

Note the significant gain in absolute error.

>For details and theory behind all the quadrature formulas that we have explored, a good reference is _chapter 3_ (Numerical Differentiation and Integration) of Walter Gautchi's _Numerical Analysis_.

### Functions with singularities on bounded domains

The second case of integration is that of definite integrals on a finite interval `[a,b]` of functions with singularities.  We contemplate two cases: _weighted functions_, and _generic functions with singularities_.

#### Weighted Functions

Weighted functions can be realized as products of the kind $f(x) w(x)$ for some smooth function $f(x)$ with a non-negative weight function $w(x)$ containing singularities.  An illustrative example is given by $\cos(\pi x/2)/\sqrt{x}$.  We may regard this case as the product of $\cos(\pi x/2)$ with $w(x)=1/\sqrt{x}$.  The weight presents a single singularity at $x=0$, and is smooth otherwise.

The usual way to treat these integrals is by means of  _weighted Gaussian quadrature formulas_.

For example, to perform _principal value integrals_ of functions of the form $f(x)/(x-c)$, we issue `quad` with the arguments `weight='cauchy'` and `wvar=c`.  This calls the routine `QAWC` from `QUADPACK`.

Let us experiment with the Fresnel-type sine integral of $g(x) = \sin(x)/x$ on the interval $[-1,1]$, and compare with `romberg`:

In [ ]:
value, absolute_error = quad(f, -1, 1, weight='cauchy', wvar=0)
print value

print romberg(g, -1, 1)

For the case of integrals of functions of the form $f(x) (x-a)^\alpha (b-x)^\beta$, where $a$ and $b$ are the endpoints of the domain of integration, and both $\alpha$, $\beta$ are greater than -1, we issue `quad` with the arguments `weight='alg'` and `wvar=(alpha, beta)`.  This calls the routine `QAWS` from `QUADPACK`.  

Let us experiment with the Fresnel-type cosine integral of $g(x) = \cos(\pi x/2)/\sqrt{x}$ on the interval $[0,1]$, and compare with `quadrature`: 

In [ ]:
def f(x): return np.cos(np.pi * x * 0.5)
    
def g(x): return np.cos(np.pi * x * 0.5) / np.sqrt(x)

value, absolute_error = quad(f, 0, 1, weight='alg', wvar=(-0.5,0))
print value

print quadrature(g, 0, 1)

If the weight function has the form $w(x) = (x-a)^\alpha  (b-x)^\beta  \ln(x-a)$, or $w(x) = (x-a)^\alpha  (b-x)^\beta \ln(b-x)$, or $w(x) = (x-a)^\alpha  (b-x)^\beta  \ln(x-a)  \ln(b-x)$, we issue `quad` with the arguments `weight='alg-loga'`, `weight='alg-logb'`, or `weight='alg-log'` respectively, and on each case, `wvar=(alpha, beta)`.

For example, for the function $g(x) = x^2  \ln(x)$ on the interval $[0,1]$, we could issue:

In [ ]:
def f(x): return x**2
    
def g(x): return x**2 * np.log(x)

value, absolute_error = quad(f, 0, 1, weight='alg-loga', wvar=(0,0))
print value

#### General functions with singularities

In general, we may be handling functions with singularities that do not conform to the nice form $f(x) w(x)$.  In that case, if we are aware of the locations of the singularities, we may indicate so to the integrator `quad` with the optional argument `points`.   In this case, `quad` calls the routine `QAGP` in `QUADPACK`.

For example, for the function $g(x) = \lfloor x\rfloor  \ln(x)$ that observes a singularity on each integer number, to integrate in the interval $[1,8]$ we could issue:

In [ ]:
def g(x): return np.floor(x) * np.log(x)

quad(g, 1, 8, points=np.arange(8)+1)

Compare to a simple `quad` computation without indicating singularities:

In [ ]:
quad(g, 1, 8)

In [ ]:
def f(x): return 2 * np.exp(-x**2) / np.sqrt(np.pi)

value, absolute_error = quad(f, 0, np.inf)
print value                                  # Compare with similar result in previous section

In [ ]:
def f(x): return np.sin(x)/x

value, absolute_error = quad(f, 0, np.inf)
print value

In [ ]:
print integrate(Sin(x)/x, (x, 0, oo))

For the case of oscillatory integrals in unbounded domains, besides issuing `quad` with the argument `weight='cos'` or `weight='sin'`, and the corresponding `wvar` parameter, we may also place an upper bound on the number of cycles to use internally.  We do so by setting the optional argument `limlst` to the desired bound.  It is usually a good idea to set it to something larger than 3.  

For example, for the Fourier-like integral of the sinc function in the interval $[1, \infty]$, we could issue

In [ ]:
def f(x): return 1./x

print quad(f, 1, np.inf, weight='sin', wvar=1, limlst=5)
    
print quad(f, 1, np.inf, weight='sin', wvar=1, limlst=50)

In [ ]:
def f(x, y): return np.exp(-x**2 - y**2)

from scipy.integrate import dblquad

dblquad(f, 0, np.inf, lambda x:0, lambda x:np.inf)